In [22]:
import argparse
import copy
import json
import os
import sys

try:
    import apex
except:
    print("No APEX!")
import numpy as np
import torch
from torch import nn
import yaml
from det3d import torchie
from det3d.datasets import build_dataloader, build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    batch_processor,
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
    example_to_device,
)
from det3d.torchie.trainer import load_checkpoint
import pickle 
import time 
from matplotlib import pyplot as plt 
from det3d.torchie.parallel import collate, collate_kitti
from torch.utils.data import DataLoader
import matplotlib.cm as cm
import subprocess
import cv2
from tools.demo_utils import visual 
from collections import defaultdict

No APEX!


In [2]:
cfg = Config.fromfile('configs/nusc/pp/nusc_centerpoint_pp_02voxel_two_pfn_10sweep_demo.py')

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)

dataset = build_dataset(cfg.data.val)

data_loader = DataLoader(
    dataset,
    batch_size=1,
    sampler=None,
    shuffle=False,
    num_workers=8,
    collate_fn=collate_kitti,
    pin_memory=False,
)

checkpoint = load_checkpoint(model, './latest.pth', map_location="cpu")
model.eval()

model = model.cuda()

gpu_device = torch.device("cuda")

points_list = [] 
gt_annos = [] 
detections  = [] 

data_iter = iter(data_loader)
data_batch = next(data_iter)



Use HM Bias:  -2.19
10


In [3]:
example = example_to_device(data_batch, gpu_device, non_blocking=False)
out_box = model(example, return_loss=False)

In [4]:
# x_size 512
x_size = int((cfg.model.reader.pc_range[3] - cfg.model.reader.pc_range[0])/cfg.model.reader.voxel_size[0])
x_size

512

In [5]:
idx = example["coordinates"][:,2]*x_size+example["coordinates"][:,3]
idx = idx.cpu().numpy().astype(np.int32).reshape(-1,1)

In [6]:
max_pillars = 30000
onnx_idx_input = np.zeros((1, max_pillars,2),np.int32)

In [7]:
onnx_idx_input.shape

(1, 30000, 2)

In [8]:
onnx_idx_input[0,:idx.shape[0],1:] = idx
# pad with -1
onnx_idx_input[0,idx.shape[0]:,:] = -1

In [9]:
onnx_idx_input

array([[[     0, 129776],
        [     0, 129775],
        [     0, 129774],
        ...,
        [    -1,     -1],
        [    -1,     -1],
        [    -1,     -1]]], dtype=int32)

In [10]:
#save for trt inference
onnx_idx_input.tofile("tensorrt/data/centerpoint/idx.bin")

In [11]:
pillars_data = model.reader.features.cpu().numpy()
pillars_data.shape

(5268, 20, 10)

In [12]:
# pad zeros (1, 5268, 20, 10) to (1, 30000, 20, 10)
trt_pillars_input = np.zeros((1, max_pillars, 20, 10) ,np.float32)

In [13]:
#save for trt inference
trt_pillars_input[0,:pillars_data.shape[0],:,:] = pillars_data
trt_pillars_input.transpose(0,3,1,2).tofile("tensorrt/data/centerpoint/pillars.bin")

In [14]:
out = model.preds
torch_out = out[0]["reg"].cpu().detach().numpy()

In [15]:
torch_out.shape

(1, 128, 128, 2)

In [16]:
# get the trt_output 
trt_output = np.fromfile("./tensorrt/data/centerpoint/594.bin",np.float32).reshape(1,2,128,128).transpose(0,2,3,1)

In [17]:
# compare trt_output with torch output
np.max(np.abs(torch_out.reshape(-1) - trt_output.reshape(-1)))

0.06456113

In [18]:
idx = np.argmax(np.abs(torch_out.reshape(-1) - trt_output.reshape(-1)))

In [19]:
np.abs(torch_out.reshape(-1) - trt_output.reshape(-1))

array([0.00052282, 0.00067577, 0.00148186, ..., 0.00159258, 0.00035995,
       0.00091583], dtype=float32)

In [20]:
torch_out.reshape(-1)[idx]

1.2549908

In [21]:
trt_output.reshape(-1)[idx]

1.1904297